In [1]:
# !pip install python-keycloak
# !pip install psycopg2-binary
import csv
import requests
import psycopg2
from pytz import timezone
from datetime import datetime, timedelta
from dateutil import parser, relativedelta
from keycloak import KeycloakOpenID
from keycloak import KeycloakAdmin
import keycloak
import json
 
START_DATE = "2022-10-01";
END_DATE = "2022-10-31";
POSTGRES_USER = "SE_MOSAI"
POSTGRES_PASSWORD = 'MkoNgt#28#asd'
POSTGRES_HOST = "hklvdpsql062.hk.standardchartered.com"
POSTGRES_PORT = "6524"
POSTGRES_DB_CONSOLE = "mosaic_console"
POSTGRES_DB_AI = "ai_logistics"
POSTGRES_DB_METERING = "metering"
POSTGRES_DB_USERMGM = "usermgm"
POSTGRES_DB_MONITOR = "mosaic_monitor"
SERVER_URL="http://keycloak.keycloak/auth/"
REALM="master"
CLIENT_ID="admin-cli"
KEYCLOAK_USERNAME="keycloak"
KEYCLOAK_PASSWORD="90f8372d933d4fd1fd311f0b8cd36a3d"
APP_ADMIN_USER = "1632997"
TIME_ZONE_GMT = "Etc/GMT+7"
TIME_ZONE_IST = "Asia/Kolkata"

keycloak_openid = KeycloakOpenID(server_url=SERVER_URL,
                        client_id=CLIENT_ID,
                        realm_name=REALM,                                       
                        client_secret_key="")

token = keycloak_openid.token(KEYCLOAK_USERNAME, KEYCLOAK_PASSWORD)
access_token = token['access_token']
def get_user_activity_details(access_token):
    try:
        connection_usermgm = psycopg2.connect(user=POSTGRES_USER,
                                              password=POSTGRES_PASSWORD,
                                              host=POSTGRES_HOST,
                                              port=POSTGRES_PORT,
                                              database=POSTGRES_DB_USERMGM,
                                              options="-csearch_path=usermgm")
        result = getUsersActivity(access_token, connection_usermgm)
        return result
    except Exception as ex:
        print("Exception : ", ex)
        
def get_all_users_keycloak(token):
    url=SERVER_URL+"admin/realms/mosaic/users?briefRepresentation=true&first=0&max=500"
    auth_token = "Bearer "+token
    headers= {"Authorization": auth_token, "accept": "application/json" }
    response = requests.get(url, headers=headers)
    return response.json()

def get_all_users():
    url="http://user-management:8094/usermgm/user/getAllUsers"
    headers= {"X-Auth-Userid": "mosaic", "accept": "application/json"}
    response = requests.get(url, headers=headers)
    return response.json()

def get_users_representation(tokenn):
    #print(tokenn)
    url=SERVER_URL+"admin/realms/mosaic/users?first=0&max=500"
    auth_token = "Bearer "+tokenn
    headers= {"Authorization": auth_token, "accept": "application/json" }
    response = requests.get(url, headers=headers)
    return response.json()

# def get_users_events(userId, endDate, token):
#     url=SERVER_URL+"admin/realms/mosaic/events?dateFrom="+START_DATE+"&first=0&max=2147483647"
#     if endDate!="":
#         url=SERVER_URL+"admin/realms/mosaic/events?type=LOGIN&dateFrom="+START_DATE+"&dateTo="+endDate+"&user="+userId+"&first=0&max=2147483647"
#         print(url)
#     auth_token = "Bearer "+token
#     headers= {"Authorization": auth_token, "accept": "application/json" }
#     response = requests.get(url, headers=headers)
#     return response.json()

def get_users_events(userId, endDate, token):
    url=SERVER_URL+"admin/realms/mosaic/events?type=LOGIN&dateFrom="+START_DATE+"&dateTo="+END_DATE+"&first=0&max=2147483647"
    if endDate!="":
        url=SERVER_URL+"admin/realms/mosaic/events?type=LOGIN&dateFrom="+START_DATE+"&dateTo="+endDate+"&user="+userId+"&first=0&max=2147483647"
    auth_token = "Bearer "+token
    headers= {"Authorization": auth_token, "accept": "application/json" }
    response = requests.get(url, headers=headers)
    return response.json()

def get_users_events_trendwise(userId, startDate, endDate, token):
    url=SERVER_URL+"admin/realms/mosaic/events?type=LOGIN&type=LOGOUT&dateFrom="+startDate+"&dateTo="+endDate+"&user="+userId+"&first=0&max=2147483647"
    auth_token = "Bearer "+token
    headers= {"Authorization": auth_token, "accept": "application/json" }
    response = requests.get(url, headers=headers)
    return response.json()

def createUsersInfo(userRepresentations, usersList, allUsers):
    userInfoList = []
    for userRepresentation in userRepresentations:
        usersListStr = ' '.join([str(elem) for elem in usersList])
        if userRepresentation['id'] in usersListStr:
            userMap = {}
            userMap["username"] = userRepresentation['username']
            #userMap["email"]= userRepresentation['email'];
            userMap["name"]= userRepresentation['firstName']+" "+ userRepresentation['lastName'];
            userMap["id"] = userRepresentation['id'];
            for user in allUsers:
                if ((user['username']).casefold()== (userRepresentation['username']).casefold()):
                    userMap["group"]= ', '.join(user["groups"])
            userInfoList.append(userMap);
    return userInfoList

def getUsersActivity(token, connection_usermgm):
    eventsList = ["LOGIN", "LOGOUT"]
    usersList = set()
    eventUsers= set()
    activeUsersList = []
    responseList =[]
    userRepresentations = get_users_representation(token)
    for userRepresentation in userRepresentations:
        usersList.add(userRepresentation['id'])
    allUsers = get_all_users()
    userInfoList = createUsersInfo(userRepresentations, usersList, allUsers)
    endDate = (datetime.combine((datetime.now()+timedelta(days=1)).date(), datetime.min.time())).strftime('%Y-%m-%d')
    eventRepresentationsAllUsers = get_users_events("", "", token)
    print(eventRepresentationsAllUsers[0])
    for event in eventRepresentationsAllUsers:
        eventUsers.add(event['userId'])
    print(eventUsers)
    roleRequests = get_user_role_requests(connection_usermgm)
    inactive_users = [x for x in usersList if x not in eventUsers]
    for userId in eventUsers:
        eventRepresentations = get_users_events(userId, endDate, token)
        if userId=='7f0a2963-f4b0-4a04-bbb0-1c8b88404901':
            with open('/data/Dump.csv', 'w', newline='') as output_file:
                dict_writer = csv.DictWriter(output_file, eventRepresentations[0].keys())
                dict_writer.writeheader()
                dict_writer.writerows(eventRepresentations)
                
        activeUsersMap = {}
        activeUsersMap['id'] = userId
        lasloginTime = eventRepresentations[0]["time"]
        firstLoginTime = eventRepresentations[len(eventRepresentations)-1]["time"]            
        activeUsersMap['lastLoginTime']=lasloginTime
        activeUsersMap['firstLoginTime']=firstLoginTime
        activeUsersMap['loginCount']=len(eventRepresentations)
        activeUsersList.append(activeUsersMap)
        responseList = buildUserActivityList(userInfoList, activeUsersMap, responseList, None)
    for inactive_user in inactive_users:
        responseList = buildUserActivityList(userInfoList, {}, responseList, inactive_user)
    for response in responseList:
        team = set()
        country = set()
        model_family = set()
        for roleRequest in roleRequests:
            if (roleRequest[3]).casefold()==(response["username"]).casefold() :
                team.add(roleRequest[20])
                country.add(roleRequest[9])
                model_family.add(', '.join(roleRequest[19]))
        response["Team"]= ', '.join(team)
        response["Country"]= ', '.join(country)
        response["Model_family"] = model_family
    return responseList
    
def buildUserActivityList(userInfoList, activeUser, responseList, inactive_user):
    for userInfo in userInfoList:
        if len(activeUser) > 0 and userInfo['id'] == activeUser['id']:
            dt_object = datetime.fromtimestamp(int(str(activeUser['lastLoginTime'])[:-3]))
            #userInfo['lastLoginTime']= str(dt_object.astimezone(timezone(TIME_ZONE_IST)))
            userInfo['lastLoginTime']= (dt_object.astimezone(timezone(TIME_ZONE_IST))).strftime("%Y %m %d %H:%M:%S")
            dt_object = datetime.fromtimestamp(int(str(activeUser['firstLoginTime'])[:-3]))
            userInfo['firstLoginTime']= (dt_object.astimezone(timezone(TIME_ZONE_IST))).strftime("%Y %m %d %H:%M:%S")
            userInfo['loginCount']= activeUser['loginCount']
            responseList.append(userInfo)
        elif userInfo['id'] == inactive_user:
            userInfo['lastLoginTime']= ""
            userInfo['firstLoginTime']= ""
            userInfo['loginCount']= 0
            responseList.append(userInfo)
    return responseList

def get_user_role_requests(connection):
    try:
        cursor = connection.cursor()
        postgreSQL_select_Query = "select * from "+POSTGRES_DB_USERMGM+".user_role_request where dm_request_status=1"
        cursor.execute(postgreSQL_select_Query)
        user_role_requests = cursor.fetchall()
        return user_role_requests
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL for : ", error)
        return -1
    finally:
        cursor.close()

#=====================Template============================

def get_all_users():
    url="http://user-management:8094/usermgm/user/getAllUsers"
    headers= {"X-Auth-Userid": "mosaic", "accept": "application/json"}
    response = requests.get(url, headers=headers)
    return response.json()

def get_total_projects_for_given_user(username):
    url="http://mosaic-console-backend:80/mosaic-console-backend/secured/api/project/v1"
    headers= {"X-Auth-Userid": username,"X-Auth-Username": username, "accept": "application/json"}
    response = requests.get(url, headers=headers)
    return response.json()


def get_projects_created_count(connection, username, status):
    try:
        cursor = connection.cursor()
        postgreSQL_select_Query = "select count(id)as count from "+POSTGRES_DB_CONSOLE+".project where created_by='"+username+"' AND status='"+status+"' AND (date(created_date) >='"+START_DATE+"' AND date(created_date) <='"+END_DATE+"')"
        cursor.execute(postgreSQL_select_Query)
        no_of_projects = cursor.fetchone()
        return no_of_projects[0]
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL for '"+username+"' : ", error)
        return -1
    finally:
        cursor.close()

def get_the_total_templates_and_notebook_runs(username):
    payload = {
        "startDate":START_DATE,
        "endDate":END_DATE+" 24:00:00.000",
        "userList":[username],
        "recordCount": 1000000,
        "pageNo":1,
        "componentType":[]
    }
    url="http://mosaic-console-backend:80/mosaic-console-backend/secured/api/activity/v1/filter"
    headers= {"X-Auth-Userid": APP_ADMIN_USER,"X-Auth-Username": APP_ADMIN_USER, "accept": "application/json"}
    response = requests.post(url, headers=headers, json=payload)
    response_json = response.json()["data"]
    #notebook_runs = len([x for x in response_json if x["object_type"] == 'NOTEBOOK' and x["action"] == 'RUN'])
    temp = [x for x in response_json if x["object_type"] == 'NOTEBOOK' and x["action"] == 'RUN']
    notebook_runs = len(temp)
    template_create = len([x for x in response_json if x["object_type"] == 'TEMPLATE' and x["action"] == 'CREATE'])
    return {"notebook_runs":notebook_runs, "template_create":template_create}


def get_project_game_linkage_count(connection, username):
    try:
        cursor = connection.cursor()
        postgreSQL_select_Query = "select count(jira_id) as project_count from "+POSTGRES_DB_CONSOLE+".game_project_master where created_by = '"+username+"' AND (date(created_date) >='"+START_DATE+"' AND date(created_date) <='"+END_DATE+"')"
        cursor.execute(postgreSQL_select_Query)
        no_of_notebooks = cursor.fetchone()
        return no_of_notebooks[0]
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL for '"+username+"' : ", error)
        return -1
    finally:
        cursor.close()
        
        
def get_sas_template_count(connection, username, template_name='SAS'):
    try:
        cursor = connection.cursor()
        postgreSQL_select_Query = "SELECT count(1) AS sas_template_spwan_cnt FROM "+POSTGRES_DB_AI+".nb_template_status t_status INNER JOIN "+POSTGRES_DB_AI+".nb_docker_image template ON TRIM(t_status.template_id) = TRIM(template.id) WHERE t_status.created_by='"+username+"' AND (date(start_date) >= '"+START_DATE+"' AND date(start_date) <= '"+END_DATE+"') AND (upper(template.base_template)='"+template_name+"' OR template.base_image_id=(select id from "+POSTGRES_DB_AI+".nb_docker_image where upper(name)='"+template_name+"'))"
        cursor.execute(postgreSQL_select_Query)
        no_of_notebooks = cursor.fetchone()
        return no_of_notebooks[0]
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL for '"+username+"' : ", error)
        return -1
    finally:
        cursor.close()
        

def get_cpu_hrs_per_user(connection, username):
    try:
        cursor = connection.cursor()
        postgreSQL_select_Query = "select ROUND(CAST(SUM(CAST(current_usage->>'cpu_hours' AS FLOAT)) as numeric), 2) from "+POSTGRES_DB_METERING+".requests  where created_by='"+username+"' AND (date(created_on) >='"+START_DATE+"' AND date(created_on) <='"+END_DATE+"') group by created_by"
        cursor.execute(postgreSQL_select_Query)
        no_of_notebooks = cursor.fetchone()
        return no_of_notebooks[0]
    except (Exception, psycopg2.Error) as error:
        #print("Error while fetching data from PostgreSQL for '"+username+"' : ", error)
        return 0
    finally:
        cursor.close()
        
        
def get_total_active_templates(connection, username):
    try:
        cursor = connection.cursor()
        postgreSQL_select_Query = "select count(1) as c from "+POSTGRES_DB_AI+".nb_docker_image where created_by='"+username+"' AND (date(created_on) >= '"+START_DATE+"' AND date(created_on) <= '"+END_DATE+"')"
        cursor.execute(postgreSQL_select_Query)
        no_of_notebooks = cursor.fetchone()
        return no_of_notebooks[0]
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL for '"+username+"' : ", error)
        return -1
    finally:
        cursor.close()
        

def get_notebook_run_count(connection, username):
    try:
        cursor = connection.cursor()
        postgreSQL_select_Query = "select COUNT(1) AS C from "+POSTGRES_DB_MONITOR+".job_monitor where job_type='NOTEBOOK' and job_createdby='"+username+"' AND (date(job_createdts) >= '"+START_DATE+"' AND date(job_createdts) <= '"+END_DATE+"')"
        cursor.execute(postgreSQL_select_Query)
        no_of_notebooks = cursor.fetchone()
        return no_of_notebooks[0]
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL for '"+username+"' : ", error)
        return -1
    finally:
        cursor.close()       

        
def get_template_spwan_count(connection=None, username=None):
    try:
        result = {}
        cursor = connection.cursor()
        postgreSQL_select_Query = "SELECT template1.name as template_name, count(1) AS template_spwan_cnt FROM "+POSTGRES_DB_AI+".nb_template_status t_status INNER JOIN "+POSTGRES_DB_AI+".nb_docker_image template ON TRIM(t_status.template_id) = TRIM(template.id) LEFT OUTER JOIN "+POSTGRES_DB_AI+".nb_docker_image template1 ON TRIM(COALESCE(template.base_image_id, template.id)) = TRIM(template1.id) WHERE t_status.created_by='"+username+"' AND (date(start_date) >= '"+START_DATE+"' AND date(start_date) <= '"+END_DATE+"') group by template1.name"
        cursor.execute(postgreSQL_select_Query)
        records = cursor.fetchall()
        for record in records:
            result[record[0]] = record[1]
        return result
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL for '"+username+"' : ", error)
        return -1
    finally:
        cursor.close()



In [2]:

try:
    report_records = []
    all_users = get_user_activity_details(access_token)
    print("The total no of users are : ", len(all_users))
    connection_mosaic_console = psycopg2.connect(user=POSTGRES_USER,
                                                 password=POSTGRES_PASSWORD,
                                                 host=POSTGRES_HOST,
                                                 port=POSTGRES_PORT,
                                                 database=POSTGRES_DB_CONSOLE)
    connection_ai_logistics = psycopg2.connect(user=POSTGRES_USER,
                                                 password=POSTGRES_PASSWORD,
                                                 host=POSTGRES_HOST,
                                                 port=POSTGRES_PORT,
                                                 database=POSTGRES_DB_AI)
    connection_metering = psycopg2.connect(user=POSTGRES_USER,
                                                 password=POSTGRES_PASSWORD,
                                                 host=POSTGRES_HOST,
                                                 port=POSTGRES_PORT,
                                                 database=POSTGRES_DB_METERING)
    connection_monitor = psycopg2.connect(user=POSTGRES_USER,
                                                 password=POSTGRES_PASSWORD,
                                                 host=POSTGRES_HOST,
                                                 port=POSTGRES_PORT,
                                                 database=POSTGRES_DB_MONITOR)
    for user in all_users:
        report_record = {}
        username = user["username"]
        report_record['USER_NAME']=username
        report_record['NAME_OF_THE_USER']=user["name"]
        report_record['LAST_LOGIN']=user["lastLoginTime"]
        report_record['FIRST_LOGIN']=user["firstLoginTime"]
        report_record['LOGIN_COUNT']=user["loginCount"]
        report_record['TEAM']=user["Team"]
        report_record['COUNTRY']=user["Country"]
        report_record['GROUP']=user["group"]
        print("\n "+username+" : ")
       
    
        # TOTAL_ACTIVE_PROJECTS_CREATED
        total_active_projects_created = get_projects_created_count(connection_mosaic_console, username, "ACTIVE")
        report_record["TOTAL_ACTIVE_PROJECTS_CREATED"]=total_active_projects_created
        #print("Total active Projects created : "+str(total_active_projects_created))
        
        # TOTAL_INACTIVE_PROJECTS_CREATED
        total_inactive_projects_created = get_projects_created_count(connection_mosaic_console, username, "INACTIVE")
        report_record["TOTAL_PROJECTS_CREATED"] = total_active_projects_created+total_inactive_projects_created
        #print("Total inactive Projects created : "+str(total_inactive_projects_created))
        
        # OVERALL TEMPLATES & NOTEBOOKS RUN
        template_details = get_the_total_templates_and_notebook_runs(username)
        report_record["OVERALL_TEMPLATES_CREATED"] = template_details["template_create"]
        #print("Total Templates Created : "+str(template_details["template_create"]))
        report_record["TOTAL_NOTEBOOKS_SPAWNED"] = template_details["notebook_runs"]
        #print("Total Notebooks Runs : "+str(template_details["notebook_runs"]))
        
        # CPU Hrs
        total_cpu_hrs = get_cpu_hrs_per_user(connection_metering, username) 
        report_record["CPU_Hrs"] = total_cpu_hrs
        #print("The total CPU Hrs are : ", total_cpu_hrs)
        
        # Project Game Linkage
        project_game_linkage_count = get_project_game_linkage_count(connection_mosaic_console, username)   
        report_record["TOTAL_PROJECTS_LINKED_TO_GAME"] = project_game_linkage_count
        #print("Project-Game Linkage Count: ", project_game_linkage_count)

#         # Template Spwan Count
        template_spawn_count = get_template_spwan_count(connection_ai_logistics, username)  
        report_record["TEMPLATES_SPAWNED_SAS"] = template_spawn_count.get("SAS", 0)
        report_record["TEMPLATES_SPAWNED_ZEPPELIN"] = template_spawn_count.get("Zeppelin", 0)
        report_record["TEMPLATES_SPAWNED_PYTHON3.6"] = template_spawn_count.get("Python-3.6", 0)
        report_record["TEMPLATES_SPAWNED_SCIPY"] = template_spawn_count.get("Scipy", 0)
        report_record["TEMPLATES_SPAWNED_RJUPYTER"] = template_spawn_count.get("R-Jupyter", 0)
        report_record["TEMPLATES_SPAWNED_TENSORFLOW"] = template_spawn_count.get("Tensorflow", 0)
        report_record["TEMPLATES_SPAWNED_PYTHON3.7"] = template_spawn_count.get("Python-3.7", 0)
        report_record["TEMPLATES_SPAWNED_SPARK-JUPYTERLAB"] = template_spawn_count.get("spark-Jupyterlab", 0)
        report_record["TEMPLATES_SPAWNED_SPARK"] = template_spawn_count.get("Spark", 0)
        report_record["TEMPLATES_SPAWNED_RJUPYTERLAB"] = template_spawn_count.get("R-Jupyterlab", 0)
        report_record["TEMPLATES_SPAWNED_JUPYTERLAB"] = template_spawn_count.get("Jupyterlab", 0)
        report_record["TEMPLATES_SPAWNED_VSCODE-JDK11"] = template_spawn_count.get("VSCode-JDK11", 0)
        report_record["TEMPLATES_SPAWNED_VSCODE-SCALA2.12"] = template_spawn_count.get("VSCode-Scala2.12", 0)
        report_record["TEMPLATES_SPAWNED_JUPYTERLAB-3.6"] = template_spawn_count.get("Jupyterlab-3.6", 0)
        report_record["TEMPLATES_SPAWNED_JUPYTERLAB-3.7"] = template_spawn_count.get("Jupyterlab-3.7", 0)
        report_record["TEMPLATES_SPAWNED_RSTUDIO-V4.1"] = template_spawn_count.get("RStudio-4.1", 0)
        report_record["TEMPLATES_SPAWNED_RSTUDIO-V4.0"] = template_spawn_count.get("RStudio-4", 0)
        
#         # SAS Template Spwan Count
#         sas_template_count = get_sas_template_count(connection_ai_logistics, username)  
#         report_record["TOTAL_SAS_TEMPLATES_SPAWNED"] = sas_template_count
#         #print("Total SAS Template Count: ", sas_template_count)
        
        # TOTAL ACTIVE TEMPLATES
        total_active_templates = get_total_active_templates(connection_ai_logistics, username)
        report_record["TOTAL_ACTIVE_TEMPLATES"] = total_active_templates
        #print("Total Active Templates : ", total_active_templates)
        
        # OVERALL NOTEBOOK RUNS
        notebook_run_count = get_notebook_run_count(connection_monitor, username)
        report_record["OVERALL_NOTEBOOK_RUNS"] = notebook_run_count
        #print("Overall Notebook Runs : ", notebook_run_count)

        
        report_records.append(report_record)
        
    if report_records:
        with open('/data/adoption_matrix_report_userlevel_v6_prod.csv', 'w', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, report_records[0].keys())
            dict_writer.writeheader()
            dict_writer.writerows(report_records)
    
    print("DONE !!")
except Exception as ex:
      print("Exception : ", ex)
finally:
    if connection_mosaic_console:
        connection_mosaic_console.close()
        print("\nPostgres Connection 'connection_mosaic_console' is closed") 
    if connection_ai_logistics:
        connection_ai_logistics.close()
        print("\nPostgres Connection 'connection_ai_logistics' is closed") 
    if connection_metering:
        connection_metering.close()
        print("\nPostgres Connection 'connection_metering' is closed")


{'time': 1667140811698, 'type': 'LOGIN', 'realmId': 'mosaic', 'clientId': 'keycloak-gatekeeper', 'userId': '941378af-dd02-4598-9efd-dca51f4f2da6', 'sessionId': '61e035f8-f9a1-459b-ae56-4779c4f5b553', 'ipAddress': '10.245.7.0', 'details': {'auth_method': 'openid-connect', 'token_id': 'd1d29620-2888-4b08-a176-44f1cb8dcfd1', 'grant_type': 'password', 'refresh_token_type': 'Refresh', 'scope': 'email profile', 'refresh_token_id': '7521e40a-e9e6-4520-9477-6efe2c45a590', 'client_auth_method': 'client-secret', 'username': 'svc_leap_mosaic'}}
{'1459e21e-c62c-4cdf-aeb5-2b25f6fa92ef', '3ae99c9b-799e-4187-8ba3-9e112b298283', 'c4201283-2efc-4796-8c29-bff9f0c53737', 'ddea4bb0-85b5-4e32-a08d-97f9eb0891f0', '54cd9c05-f5ec-44e1-a937-16f6736ef0ca', '1b74daa4-fe03-491a-98f4-6ef87d01c1df', 'e0ab7fa6-889c-4f25-aa71-f77a64f95a6c', 'f1f19adf-87dd-4a60-8a20-175c938593a4', 'b2d5f112-0511-426f-8785-0e6e9174c1d1', '656697af-bdd8-4d7a-b616-13b5b2f08684', 'abc887d1-d25b-4861-aa70-5c87096299e8', '9dce42f2-6174-4924


 1117738 : 

 1665516 : 

 1443509 : 

 1519998 : 

 1600591 : 

 1656802 : 

 1531642 : 

 1516072 : 

 1505480 : 

 1665514 : 

 1650063 : 

 1623681 : 

 1663940 : 

 1632684 : 

 1627713 : 

 1655328 : 

 1642506 : 

 1650641 : 

 1613042 : 

 1536436 : 

 1636934 : 

 1617516 : 

 1617324 : 

 1624454 : 

 1524004 : 

 1654223 : 

 8187000 : 

 1631544 : 

 1632501 : 

 1529501 : 

 1642810 : 

 1634701 : 

 1304224 : 

 1624129 : 

 1552831 : 

 1383166 : 

 1574959 : 

 1569690 : 

 1470671 : 

 1598748 : 

 1606901 : 

 1628533 : 

 1647480 : 

 1495466 : 

 1645993 : 

 1623729 : 

 1546829 : 

 1522291 : 

 1624167 : 

 1656214 : 

 1626818 : 

 1363360 : 

 1634242 : 

 1482462 : 

 1652906 : 

 1182119 : 

 1143771 : 

 1600679 : 

 1379297 : 

 1624110 : 

 1624085 : 

 1610289 : 

 1634945 : 

 1506384 : 

 1420232 : 

 1557077 : 

 1502203 : 

 1570924 : 

 daguser : 

 1666401 : 

 1628237 : 

 1647256 : 

 1626317 : 

 1663094 : 

 1551009 : 

 1633587 : 

 1618084 : 

In [16]:
!pip freeze | grep python-keycloak

python-keycloak==2.6.0


In [15]:
!pip install python-keycloak -i https://artifactory.global.standardchartered.com/artifactory/api/pypi/pypi/simple/

Looking in indexes: https://artifactory.global.standardchartered.com/artifactory/api/pypi/pypi/simple/
     |████████████████████████████████| 51kB 262kB/s eta 0:00:011
     |████████████████████████████████| 61kB 19.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 25.1MB/s eta 0:00:01
     |████████████████████████████████| 143kB 69.7MB/s eta 0:00:01
     |████████████████████████████████| 71kB 22.4MB/s eta 0:00:01
     |████████████████████████████████| 163kB 68.8MB/s eta 0:00:01
     |████████████████████████████████| 143kB 66.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 24.8MB/s eta 0:00:01
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.5.3 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.6.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.21.6 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement six~=1.15.0,